In [44]:
#Introduction

In [92]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import xlrd 

%matplotlib inline
os.getcwd()

'C:\\Users\\krist\\Desktop\\Introduction to Business Analytics\\Project'

In [117]:
names=['Station Name', 'MAC Address', 'Org Name', 'Start Date', 'Start Time Zone', 
       'End Date', 'End Time Zone', 'Transaction Date', 'Total Duration', 'Charging Time', 
       'Energy (kWh)', 'GHG Savings (kg)', 'Gasoline Savings (gallons)', 
       'Port Type', 'Port Number', 'Plug Type', 'EVSE ID', 'Address 1', 'City', 'State/Province', 'Postal Code', 
       'Country', 'Latitude', 'Longitude', 'Currency', 'Fee', 'Ended By', 'Plug In Event Id', 'Driver Postal Code', 
       'User ID', 'County', 'System S/N', 'Model Number']

f = pd.read_csv('EVChargingStationUsage.csv', header = 0, names = names)

num_rows = len(f)
print(f"Number of rows: {num_rows}")

shape = f.shape
print(f"Shape: {shape}")

print(f.dtypes)
print(f.columns.tolist())

f.head()

Number of rows: 259415
Shape: (259415, 33)
Station Name                   object
MAC Address                    object
Org Name                       object
Start Date                     object
Start Time Zone                object
End Date                       object
End Time Zone                  object
Transaction Date               object
Total Duration                 object
Charging Time                  object
Energy (kWh)                  float64
GHG Savings (kg)              float64
Gasoline Savings (gallons)    float64
Port Type                      object
Port Number                     int64
Plug Type                      object
EVSE ID                       float64
Address 1                      object
City                           object
State/Province                 object
Postal Code                     int64
Country                        object
Latitude                      float64
Longitude                     float64
Currency                       object
Fee    

C:\Users\krist\AppData\Local\Temp\ipykernel_3916\2542214927.py:8: DtypeWarning: Columns (29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  f = pd.read_csv('EVChargingStationUsage.csv', header = 0, names = names)


,Station Name,MAC Address,Org Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date,Total Duration,Charging Time,...,Longitude,Currency,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/29/2011 20:17,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,...,-122.160309,USD,0.0,Plug Out at Vehicle,3,95124.0,3284.0,NaN,NaN,NaN
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 0:00,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,...,-122.160309,USD,0.0,Customer,4,94301.0,4169.0,NaN,NaN,NaN
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 8:16,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,...,-122.160309,USD,0.0,Plug Out at Vehicle,5,94301.0,4169.0,NaN,NaN,NaN
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 14:51,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,...,-122.160309,USD,0.0,Customer,6,94302.0,2545.0,NaN,NaN,NaN
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 18:51,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,...,-122.160309,USD,0.0,Plug Out at Vehicle,7,94043.0,3765.0,NaN,NaN,NaN


In [119]:
def process_station_name(station_name):
    # Remove all spaces
    station_name = station_name.replace(" ", "")

    # Splits the data at the / and only retreives what comes after and removes all spaces
    if '/' in station_name:
        # Split at the '/' and return the part after it if it exists
        parts = station_name.split("/")
        if len(parts) > 1:  # Check if there is a second part
            return parts[1]
    # If no '/' or only one part, return the original name
    return station_name


def process_start_date(df):
    #Converts 'Start Date' column into datetime format, and creates new timeframe & month & year value for later processing
    df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')

    # create columns in the DataFrame >> presents data differently for future use
    new_date_columns = pd.DataFrame({
        'StartTimestamp': df['Start Date'].astype(int) // 10**9,
        'SMonth': df['Start Date'].dt.month,
        'SYear': df['Start Date'].dt.year,
    })
  
    # DUPLICATION STOPPER!! Otherwise datasets is ever growing everytime this is run
    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index of Start date to insert new columns right behind
    start_date_index = df.columns.get_loc('Start Date') + 1
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df


def convert_decimal_dates(value):
    # both the decimal and the normal date are str originally 
    # this identifies the "hidden" float aka serial number so that neither one nor the other gets converted to NaT during to_datetime 
    if isinstance(value, str):
        try:
            # Attempt to convert to float to check if it's a serial date
            float_value = float(value)
            # If conversion is successful, treat it as a serial date
            return pd.to_datetime('1899-12-30') + pd.to_timedelta(float_value, unit='D')
        except ValueError:
            # If conversion fails, return the original string
            return value
    elif isinstance(value, (float, int)):
        # If it's already a float or int, convert it to datetime
        return pd.to_datetime('1899-12-30') + pd.to_timedelta(value, unit='D')
    else:
        # Handle unexpected types by returning the original value
        return value


def process_end_date(df):
    # Converts 'End Date' column into datetime format, and creates new timeframe & month & year value for later processing

    # Conversion of serial values into datetime format
    f['End Date'] = f['End Date'].apply(convert_decimal_dates)
    f['End Date'] = pd.to_datetime(f['End Date'])

    # Extract day, month, year, and time as new DataFrame columns
    new_date_columns = pd.DataFrame({
        'EndTimestamp': df['End Date'].astype(int) // 10**9,
        'EMonth': df['End Date'].dt.month,
        'EYear': df['End Date'].dt.year,
    })

    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index to insert new columns
    start_date_index = df.columns.get_loc('End Date') + 1

    # Insert the new date columns into the DataFrame
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df
print(f)

                          Station Name          MAC Address  \
0           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
1           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
2           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
3           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
4           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
...                                ...                  ...   
259410  PALO ALTO CA / TED THOMPSON #3  0024:B100:0002:9F81   
259411       PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233   
259412           PALO ALTO CA / MPL #4  0024:B100:0003:CD37   
259413       PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233   
259414     PALO ALTO CA / CAMBRIDGE #4  0024:B100:0003:3A0A   

                  Org Name        Start Date Start Time Zone  \
0        City of Palo Alto   7/29/2011 20:17             PDT   
1        City of Palo Alto    7/30/2011 0:00             PDT   
2        City of Palo Alto    7/30/2011 8:16       

In [121]:
# Apply the function to the 'Station Name' column
f['Station Name'] = f['Station Name'].apply(process_station_name)
print("Cleaning Station Name column is systematised")
f = process_start_date(f)
print("Start date converted to datetime64[ns] format and new columns generated")
f = process_end_date(f)
print("End date converted to datetime64[ns], with serial values accounted for, format and new columns generated")

Cleaning Station Name column is systematised


TypeError: Converting from datetime64[ns] to int32 is not supported. Do obj.astype('int64').astype(dtype) instead

In [123]:
## Replaces the serial numbers with date and makes the right columns etc

def process_transaction_datev1(df):
    """
    Converts 'End Date' column in the DataFrame to datetime format, handling serial numbers,
    extracts day, month, year, and time, and replaces the original 'End Date'
    column with the new columns.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the 'End Date' column.

    Returns:
    pd.DataFrame: The modified DataFrame with new date columns.
    """

    # Conversion of serial values into datetime format
    f['Transaction Date'] = f['Transaction Date'].apply(convert_decimal_dates)
    f['Transaction Date'] = pd.to_datetime(f['Transaction Date'])

    # Apply conversion to each value in 'End Date' column

    # Extract day, month, year, and time as new DataFrame columns
    new_date_columns = pd.DataFrame({
        'TransactionTimestamp': df['Transaction Date'].astype(int) // 10**9,
        'TMonth': df['Transaction Date'].dt.month,
        'TYear': df['Transaction Date'].dt.year,
    })

    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index to insert new columns
    start_date_index = df.columns.get_loc('Transaction Date') + 1

    # Insert the new date columns into the DataFrame
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df

# Apply the function
f = process_transaction_datev1(f)


print(f['Transaction Date'])
print(f.loc[55276,'Transaction Date'])
print(f.loc[55277,'Transaction Date'])

TypeError: Converting from datetime64[ns] to int32 is not supported. Do obj.astype('int64').astype(dtype) instead

In [125]:
## Replaces the serial numbers with date and makes the right columns etc
## ALSO COPY THE DATA FROM END DATE INTO THE ROWS WITH NaT

def process_transaction_datev2(df):
    """
    Converts 'End Date' column in the DataFrame to datetime format, handling serial numbers,
    extracts day, month, year, and time, and replaces the original 'End Date'
    column with the new columns.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the 'End Date' column.

    Returns:
    pd.DataFrame: The modified DataFrame with new date columns.
    """

    # Conversion of serial values into datetime format
    f['Transaction Date'] = f['Transaction Date'].apply(convert_decimal_dates)
    f['Transaction Date'] = pd.to_datetime(f['Transaction Date'])
    f['Transaction Date'] = f['Transaction Date'].fillna(pd.to_datetime(f['End Date']))

    # Apply conversion to each value in 'End Date' column

    # Extract day, month, year, and time as new DataFrame columns
    new_date_columns = pd.DataFrame({
        'TransactionTimestamp': df['Transaction Date'].astype(int) // 10**9,
        'TMonth': df['Transaction Date'].dt.month,
        'TYear': df['Transaction Date'].dt.year,
    })

    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index to insert new columns
    start_date_index = df.columns.get_loc('Transaction Date') + 1

    # Insert the new date columns into the DataFrame
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df

# Apply the function
f = process_transaction_datev2(f)


print(f['Transaction Date'])
print(f.loc[55276,'Transaction Date'])
print(f.loc[55277,'Transaction Date'])

ValueError: time data "42845.81041" doesn't match format "%m/%d/%Y %H:%M", at position 116186. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [127]:
print(f.dtypes)

Station Name                          object
MAC Address                           object
Org Name                              object
Start Date                    datetime64[ns]
Start Time Zone                       object
End Date                              object
End Time Zone                         object
Transaction Date              datetime64[ns]
Total Duration                        object
Charging Time                         object
Energy (kWh)                         float64
GHG Savings (kg)                     float64
Gasoline Savings (gallons)           float64
Port Type                             object
Port Number                            int64
Plug Type                             object
EVSE ID                              float64
Address 1                             object
City                                  object
State/Province                        object
Postal Code                            int64
Country                               object
Latitude  

In [58]:
#Data pre-processing - Dataset Info
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259415 entries, 0 to 259414
Data columns (total 33 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Station Name                     259415 non-null  object 
 1   MAC Address                      259415 non-null  object 
 2   Org Name                         259415 non-null  object 
 3   Start Date                       259415 non-null  object 
 4   Start Time Zone                  259415 non-null  object 
 5   End Date                         259415 non-null  object 
 6   End Time Zone                    259415 non-null  object 
 7   Transaction Date (Pacific Time)  259206 non-null  object 
 8   Total Duration (hh:mm:ss)        259415 non-null  object 
 9   Charging Time (hh:mm:ss)         259415 non-null  object 
 10  Energy (kWh)                     259415 non-null  float64
 11  GHG Savings (kg)                 259415 non-null  float64
 12  Ga

,Energy (kWh),GHG Savings (kg),Gasoline Savings (gallons),Port Number,EVSE ID,Postal Code,Latitude,Longitude,Fee,Plug In Event Id,Driver Postal Code,System S/N
count,259415.000000,259415.000000,259415.000000,259415.000000,1.804670e+05,259415.000000,259415.000000,259415.000000,259415.000000,2.594150e+05,251013.000000,1.804670e+05
mean,8.544277,3.588596,1.072306,1.591627,1.462973e+05,94302.315768,37.439565,-122.151264,1.097481,1.188685e+08,94170.308255,1.443790e+11
std,7.185462,3.017894,0.901776,0.491534,2.429677e+05,1.941657,0.009621,0.015382,1.934408,4.057750e+08,4249.837126,2.483280e+10
min,0.010000,0.004000,0.001000,1.000000,3.792000e+03,94301.000000,37.422104,-122.163239,0.000000,1.000000e+00,1013.000000,1.023100e+11
25%,3.784000,1.589000,0.475000,1.000000,4.358300e+04,94301.000000,37.428318,-122.162170,0.000000,1.573000e+03,94087.000000,1.231100e+11
50%,6.868634,2.885000,0.862000,2.000000,1.074270e+05,94301.000000,37.444489,-122.158272,0.000000,3.573000e+03,94304.000000,1.427410e+11
75%,11.458340,4.813000,1.438000,2.000000,1.745590e+05,94303.000000,37.446613,-122.145699,1.640000,6.340000e+03,94804.000000,1.705410e+11
max,97.360000,40.891000,12.219000,2.000000,5.535441e+06,94306.000000,37.449375,-122.112480,84.560000,1.609436e+09,99354.000000,2.002410e+11


In [82]:
empty_cells = df.isnull()
count_row = 0
count_column = 0
for i in range(shape[0]):
    if empty_cells.iloc[i].any():
        count_row +=1
    for j in range(shape[1]):
        if empty_cells.iloc[j].any():
            count_
        
print(count_row + count_column)

KeyboardInterrupt: 

In [ ]:
#Data Analysis
from sklearn import cluster

In [7]:
#Visualization

In [9]:
#Mandatory Component

In [11]:
#Exploratory Component

In [ ]:
#Conclusions